<a href="https://colab.research.google.com/github/Mehjabin08/Academics/blob/main/Weather_prediction_model_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive



In [ ]:

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, classification_report, roc_curve, auc
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Step 1: Load the dataset
file_path = '/content/drive/MyDrive/AI lab/weather_prediction_dataset.csv'
df = pd.read_csv(file_path)

# Step 2: Drop rows with missing values (optional but useful for clean training)
df.dropna(inplace=True)

# Step 3: Predict next day's temperature for BASEL
# Shift BASEL_temp_mean to create "tomorrow's" target
df['BASEL_temp_mean_next'] = df['BASEL_temp_mean'].shift(-1)

# Remove the last row since it now has a NaN target
df = df[:-1]

# Step 4: Define features and target
X = df.drop(columns=['BASEL_temp_mean_next'])  # all columns except the target
y = df['BASEL_temp_mean_next']

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 6: Train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step 7: Predict and evaluate
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("Model Performance:")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R² Score: {r2:.2f}")



Model Performance:
MAE: 1.36
RMSE: 1.79
R² Score: 0.94


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_auc_score

# Load your dataset
file_path = '/content/drive/MyDrive/AI lab/weather_prediction_dataset.csv'
df = pd.read_csv(file_path)

# Example: create tomorrow's harmful label based on tomorrow's features shifted from today's rows
# Here, assuming harmful if tomorrow's precipitation (BASEL_precipitation) > 1.0 mm or temp_max < 5 Celsius (example thresholds)
df['Harmful_Tomorrow'] = ((df['BASEL_precipitation'].shift(-1) > 1.0) | (df['BASEL_temp_max'].shift(-1) < 5)).astype(int)

# Drop last row as it will have NaN after shifting
df = df.dropna()

# Select features to predict tomorrow's weather - today's weather features
feature_cols = ['BASEL_cloud_cover', 'BASEL_humidity', 'BASEL_pressure', 'BASEL_global_radiation',
                'BASEL_precipitation', 'BASEL_sunshine', 'BASEL_temp_mean', 'BASEL_temp_min', 'BASEL_temp_max']

X = df[feature_cols]
y = df['Harmful_Tomorrow']

# Split dataset to training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict on test data
y_pred = model.predict(X_test)
y_pred_prob = model.predict_proba(X_test)[:, 1]

# Evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)

print("Model Evaluation:")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"ROC AUC Score: {roc_auc:.2f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Model Evaluation:
Accuracy: 0.88
Precision: 0.82
Recall: 0.54
F1 Score: 0.65
ROC AUC Score: 0.81

Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.97      0.93       583
           1       0.82      0.54      0.65       148

    accuracy                           0.88       731
   macro avg       0.86      0.76      0.79       731
weighted avg       0.88      0.88      0.87       731



In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(class_weight='balanced', random_state=42)


In [ ]:
# Step 1: Train the model first
model.fit(X_train, y_train)

# Step 2: Then make predictions
y_proba = model.predict_proba(X_test)[:, 1]
y_pred = (y_proba >= 0.3).astype(int)  # Lower threshold to improve recall


In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)


In [ ]:
# Count the number of samples in each class
count_of_class_0 = sum(y_train == 0)  # Not harmful
count_of_class_1 = sum(y_train == 1)  # Harmful
from xgboost import XGBClassifier

model = XGBClassifier(scale_pos_weight=(count_of_class_0 / count_of_class_1), use_label_encoder=False, eval_metric='logloss')


In [ ]:
# 1. Count class distribution
count_of_class_0 = sum(y_train == 0)  # Non-harmful class
count_of_class_1 = sum(y_train == 1)  # Harmful class

# 2. Initialize XGBoost model with class weight balancing
model = XGBClassifier(
    scale_pos_weight=(count_of_class_0 / count_of_class_1),  # Tells model to penalize minority class more
    use_label_encoder=False,
    eval_metric='logloss'
)

# 3. Train the model
model.fit(X_train, y_train)

# 4. Get predicted probabilities for the positive (harmful) class
y_proba = model.predict_proba(X_test)[:, 1]  # Column 1 = class 1 probability

# 5. Apply a lower threshold to boost recall for harmful class
y_pred = (y_proba >= 0.3).astype(int)  # Predict harmful if probability ≥ 0.3


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:14:20] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.91      0.89      0.90       583
           1       0.60      0.64      0.62       148

    accuracy                           0.84       731
   macro avg       0.75      0.77      0.76       731
weighted avg       0.85      0.84      0.84       731



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import numpy as np

# 1. Train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
rf_model.fit(X_train, y_train)

# 2. Predict probabilities for test set
y_proba = rf_model.predict_proba(X_test)[:, 1]

# 3. Adjust threshold to improve recall for class 1 (harmful)
threshold = 0.3  # You can try 0.25, 0.35, etc.
y_pred = (y_proba >= threshold).astype(int)

# 4. Evaluate the model
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.90      0.92      0.91       583
           1       0.65      0.61      0.63       148

    accuracy                           0.85       731
   macro avg       0.78      0.76      0.77       731
weighted avg       0.85      0.85      0.85       731



In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'class_weight': ['balanced']
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, scoring='recall')
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
